In [1]:
import numpy as np
import pandas as pd

rng = np.random.default_rng(0)
from glmnet.cox import CoxFamily, CoxLM
from glmnet.glm import GLMControl
import rpy2
%load_ext rpy2.ipython

In [7]:
n, p = 100, 10
status = rng.choice([0, 1], size=n)
start = rng.integers(0, 5, size=n)
event = start + rng.integers(1, 5, size=n) 
event_data = pd.DataFrame({'event':event,
                           'status':status,
                           'start':start})
X = rng.standard_normal((n, p))
event_data

,event,status,start
0,7,0,3
1,3,0,0
2,5,0,2
3,2,1,1
4,5,1,1
...,...,...,...
95,5,0,3
96,4,1,1
97,2,1,1
98,6,0,4


## There are clearly lots of ties

In [9]:
np.unique(event).shape, n

((8,), 100)

In [3]:
%%R -i event,status,X,start -o C1,C2
library(survival)
Y = Surv(start, event, status)
M1 = coxph(Y ~ X, ties='breslow')
M2 = coxph(Y ~ X, ties='efron')
C1 = coef(M1)
C2 = coef(M2)

In [4]:
cox_fam1 = CoxFamily(start_col='start', tie_breaking='breslow')
G1 = CoxLM(family=cox_fam1, control=GLMControl(epsnr=1e-10))
np.linalg.norm(G1.fit(X, event_data).coef_ - C1) / np.linalg.norm(C1)

1.0921154605630758e-06

In [5]:
cox_fam2 = CoxFamily(start_col='start', tie_breaking='efron')
G2 = CoxLM(family=cox_fam2, control=GLMControl(epsnr=1e-10))
np.linalg.norm(G2.fit(X, event_data).coef_ - C2) / np.linalg.norm(C2)

2.7781670297805405e-06

In [6]:
np.linalg.norm(C1-C2) / np.linalg.norm(C1)

0.18687983413212678